In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai.script import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.distributed import *
from fastai.callbacks.tracker import *

torch.backends.cudnn.benchmark = True

import time

In [3]:
import fastai;fastai.__version__ #safety check

'1.0.55'

In [4]:
import torch; torch.__version__ #safety check

'1.0.1.post2'

In [5]:
from ranger import Ranger

In [6]:
from mxresnet import *

Mish activation loaded...
Mish activation loaded...


In [7]:
path = untar_data(URLs.IMAGEWOOF_160); path

PosixPath('/home/ubuntu/.fastai/data/imagewoof-160')

In [8]:
tfms = ([

        flip_lr(p=0.5)#,
        #brightness(change=(0.4,0.6)),
        #contrast(scale=(0.7,1.3)),
        #cutout(n_holes=(2,40),length=(5,30),p=.25)

    ], [])

In [9]:
bs=64
size=128

In [10]:
data = (ImageList.from_folder(path)
        .split_by_folder(valid='val')
        .label_from_folder()
        .transform(tfms=tfms,size=size) 
        .databunch(bs=bs, num_workers=8)  #windows 10 users - num_workers may need to be set to 1 or 0 (if you get pickle fork error)
        .presize(size, scale=(0.35, 1))
        .normalize(imagenet_stats))

In [11]:
optar = partial(Ranger)

In [12]:
learn = Learner(data, mxresnet50(), metrics=[accuracy], wd=1e-3,
                opt_func=optar,
                 bn_wd=False, true_wd=True,
                loss_func = LabelSmoothingCrossEntropy())

In [13]:
learn.callback_fns += [
        partial(ShowGraph)#,
        #partial(SaveModelCallback, name='model-novotest-1')
   ]

In [14]:
n = len(learn.data.train_dl)
anneal_start = int(n*5*0.7)

In [18]:
lr = 1e-2

In [17]:
learn.purge;learn.destroy

<bound method Learner.destroy of Learner(data=ImageDataBunch;

Train: LabelList (12454 items)
x: ImageList
Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128)
y: CategoryList
n02093754,n02093754,n02093754,n02093754,n02093754
Path: /home/ubuntu/.fastai/data/imagewoof-160;

Valid: LabelList (500 items)
x: ImageList
Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128)
y: CategoryList
n02093754,n02093754,n02093754,n02093754,n02093754
Path: /home/ubuntu/.fastai/data/imagewoof-160;

Test: None, model=MXResNet(
  (0): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Mish()
  )
  (1): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [19]:
res = []
num_epoch=5
for x in range(5):
    
    learn = Learner(data, mxresnet50(c_out=10), wd=1e-2, opt_func=optar,
                 metrics=[accuracy],
                 bn_wd=False, true_wd=True,
                 loss_func=LabelSmoothingCrossEntropy())
  #n = len(learn.data.train_dl)
  #anneal_start = int(n*5*0.7)
    phase0 = TrainingPhase(anneal_start).schedule_hp('lr', lr)
        
    phase1 = TrainingPhase(n*5 - anneal_start).schedule_hp('lr', lr, anneal=annealing_cos)
    phases = [phase0, phase1]
    sched = GeneralScheduler(learn, phases)
    learn.callbacks.append(sched)
    learn.fit(num_epoch)
  
    loss, acc = learn.validate()
    print(acc.numpy())

    res.append(acc.numpy())


0.708


In [29]:
#results:
print("Average Results:")
print(np.mean(res))
print()
print('Per run results:')
for item in res:
    print(item)
print()
print("std dev: ",np.std(res))

Average Results:
0.6956

Per run results:
0.702
0.68
0.692
0.696
0.708

std dev:  0.009499478


0.702
0.68
0.692
0.696
0.708
